---
layout: single
title: 'Create rasters from SMAP data in Python'
date: 2016-06-28
authors: [Matt Oakley, Zach Schira]
category: [tutorials]
excerpt: 'This tutorial demonstrates how to access SMAP data, and how to generate raster output from this data.'
sidebar:
  nav:
author_profile: false
comments: true
lang: [python]
lib: []
---

The National Snow and Ice Data Center hosts soil moisture data (from the NASA Soil Moisture Active Passive project, described [here](https://nsidc.org/data/smap), and hereafter referred to as SMAP) which is provided in Several different formats. 

### Note: this tutorial has been deprecated

For accessing SMAP data with python, we recommend the [smap_io](https://github.com/TUW-GEO/smap_io/tree/master/tests) library. 